# RL for Stock Trading

This notebook has been cleaned up and gotten comments added to it by AI. Code was also written with help from AI. 

In [ ]:
# imports that we need to read the data and run simple eda
import torch
import numpy as np
import pandas as pd
path = "/home/tjb73/palmer_scratch/otdhcnysozsxek2a.csv"
test_path = "/home/tjb73/palmer_scratch/test"
train_path = "/home/tjb73/palmer_scratch/train_path"

In [ ]:
# load in the data and understand some of it
TESTING = True
SMALL_DS = True

# load in the df with the data
df = pd.read_csv(
    path, 
    parse_dates=["date"],
    nrows=(200000 if TESTING else 2000000) if SMALL_DS else 10_000_000,
    low_memory=False,  # Suppress mixed type warning
    dtype={'SYM_SUFFIX': str}  # Explicitly handle mixed type column
) 

# show info about the df
display(df.head())
print("\nrows:", len(df), "| columns:", df.shape[1])
# df.info(show_counts=True)

# the lookback is the number of days to look back for the target return
LOOKBACK = 25 if not TESTING else 10


In [ ]:
# Clean and feature engineering script with debug prints
import pandas as pd, numpy as np, gc, os
from pandas.api.types import is_sparse, is_string_dtype
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from tqdm.notebook import tqdm

print("0. Starting feature engineering script")

# 1. Define constants
print("1. Defining constants")
ID_COLS = ["PERMNO","PERMCO","CUSIP","NCUSIP","ISSUNO","TSYMBOL","COMNAM",
           "NAMEENDT","SHRCLS","DCLRDT","DLPRC","DLRET","DLRETX","DLAMT",
           "DLSTCD","DLPDT","NEXTDT","PAYDT","RCRDDT","NWPERM","ACPERM",
           "ACCOMP"]
CATEGORICALS = ["SHRCD","EXCHCD","HEXCD","SICCD","HSICCD","HSICMG","HSICIG",
                "NAICS","PRIMEXCH","TRDSTAT","SECSTAT"]
PRICE_COLS = ["PRC","OPENPRC","ASKHI","BIDLO","BID","ASK"]
NUMERIC_KEEP = ["VOL","NUMTRD","SHROUT","CFACPR","CFACSHR",
                "VWRETD","VWRETX","EWRETD","EWRETX","SPRTRN"]

# 2. Prepare input dataframe
print("2. Preparing input dataframe")
df.columns = df.columns.str.upper()
print("  Columns uppercased")

# 2a. Compute target return
print("  Computing TARGET_RET")
df["RET"]  = pd.to_numeric(df["RET"],  errors="coerce")
df["RETX"] = pd.to_numeric(df["RETX"], errors="coerce")
df["TARGET_RET"] = df["RET"].fillna(df["RETX"]).fillna(0.0)
df.drop(columns=["RET","RETX"], inplace=True)
print("  TARGET_RET done, dropped RET/RETX")

# 2b. Clean PRICE_COLS
print("  Cleaning price columns")
for c in PRICE_COLS:
    if c in df:
        print(f"  Processing {c}")
        s = pd.to_numeric(df[c], errors="coerce")
        s[s <= 0] = np.nan
        df[c] = s

# 2c. Create price ratios
print("  Creating price ratio columns")
for base in ("OPENPRC","ASKHI","BIDLO","ASK","BID"):
    if base in df and "PRC" in df:
        col = f"{base}_RATIO"
        print(f"  {col}")
        df[col] = df[base] / df["PRC"]

# 2d. Clean other numeric columns
print("  Cleaning other numeric columns")
num_cols = [c for c in NUMERIC_KEEP if c in df]
for c in num_cols:
    print(f"  {c}")
    s = pd.to_numeric(df[c], errors="coerce")
    s[s <= 0] = np.nan
    df[c] = s
df[num_cols] = df[num_cols].fillna(0)
print("  Filled NaNs with 0 for numeric keep columns")

ratio_cols = [c for c in df if c.endswith("_RATIO")]
df[ratio_cols] = df[ratio_cols].fillna(0)
print("  Filled NaNs with 0 for ratio columns")

# 3. One-hot encode categoricals
print("3. One-hot encoding categoricals")
enc = OneHotEncoder(sparse_output=True, handle_unknown="ignore")
cat_sparse = enc.fit_transform(df[CATEGORICALS].fillna("UNK").astype(str))
cat_df = pd.DataFrame.sparse.from_spmatrix(
    cat_sparse, index=df.index,
    columns=enc.get_feature_names_out(CATEGORICALS)
)
df.drop(columns=CATEGORICALS, inplace=True)
gc.collect()
print("  One-hot encoding complete, dropped original categoricals")

# 4. Scale numeric features
print("4. Scaling numeric features")
scale_cols = num_cols + ratio_cols
df[scale_cols] = RobustScaler().fit_transform(df[scale_cols]).astype("float32")
print(f"  Scaled columns: {len(scale_cols)} features")

# 4b. Combine with keep_cols
keep_cols = ["DATE","TICKER","PRC","TARGET_RET"]
df = pd.concat([df[keep_cols + scale_cols], cat_df], axis=1)
print(f"  Concatenated base + engineered features; df shape now {df.shape}")

# 5. Handle remaining NaNs
print("5. Patching remaining NaNs")
dense_cols = [c for c in df.columns if df[c].dtype.kind in ("f", "i", "u")]
dense_nonsparse = [c for c in dense_cols if not is_sparse(df[c].dtype)]
dense_sparse    = [c for c in dense_cols if  is_sparse(df[c].dtype)]
print(f"  Dense non-sparse: {len(dense_nonsparse)}, sparse: {len(dense_sparse)}")

df[dense_nonsparse] = df[dense_nonsparse].fillna(df[dense_nonsparse].median())
print("  Filled NaNs in dense non-sparse with median")
df[dense_sparse]    = df[dense_sparse].fillna(0)
print("  Filled NaNs in dense sparse with 0")

for c in df.columns.difference(dense_cols):
    if is_string_dtype(df[c]) or df[c].dtype == object:
        df[c] = df[c].fillna("")
print("  Filled NaNs in string columns with empty string")

sparse_cols = [c for c in df.columns if is_sparse(df[c].dtype)]
df[sparse_cols] = df[sparse_cols].fillna(0)
print("  Ensured sparse columns have no NaNs")

# Define feature columns
FEAT_COLS = [c for c in df.columns if c not in keep_cols and df[c].dtype.kind in ("f","i","u")]
print(f"  Feature columns count: {len(FEAT_COLS)}")


In [ ]:
from tqdm.notebook import tqdm

# Convert DATE column to datetime if not already
# df["DATE"] = pd.to_datetime(df["DATE"])

# Get unique sorted dates from 1980 onwards
dates = df["DATE"]  # already datetime
mask = dates >= pd.Timestamp("1980-01-01")  # build filter
all_dates = sorted(dates.loc[mask].dropna().unique())  # unique + sort -> list

if TESTING:
    # keep only last 10% for quicker debugging
    split = int(len(all_dates) * 0.9)
    all_dates = all_dates[split:]

print(f"> Total unique dates for sampling: {len(all_dates)}")

# Split dates 85/15 for train/test
cut = int(len(all_dates) * 0.85)
train_dates = all_dates[:cut]
test_dates = all_dates[cut:]

print(f"> Train dates: {len(train_dates)}, Test dates: {len(test_dates)}")

keep_dates = set(all_dates)

# Filter dataframe if in testing mode
if TESTING:
    keep_dates = set(all_dates)
    df = df.loc[df["DATE"].isin(keep_dates)].copy()
    print(f"> Filtered df -> {df.shape[0]} rows over {len(keep_dates)} dates")

# Drop all rows before 2000
df = df[df["DATE"] >= pd.Timestamp("2000-01-01")]
print(f"> After dropping pre-2000: {df.shape[0]} rows, from {df['DATE'].min().date()} to {df['DATE'].max().date()}")

# Set multi-index on DATE and TICKER
df.set_index(["DATE", "TICKER"], inplace=True)
print("> MultiIndex on (DATE, TICKER) established")

import pickle
from pathlib import Path

# Build by_date dictionary using index-based lookup
by_date = {}
for date in tqdm(sorted(keep_dates), desc="Building by_date"):
    sub = df.loc[date, FEAT_COLS + ["TARGET_RET"]]
    by_date[date] = sub

print(f"> Built by_date lookup for {len(by_date)} days")

# Save by_date dictionary to disk
out_file = Path("by_date.pkl")
with out_file.open("wb") as f:
    pickle.dump(by_date, f)
print(f"> Saved by_date dictionary to {out_file.resolve()}")




In [ ]:
# Function to create and save samples to disk
def make_samples_to_disk(dates, out_dir, prefix):
    print(f"make_samples_to_disk: {prefix} — start")
    dates = pd.DatetimeIndex(dates).intersection(by_date.keys())
    print(f"    {len(dates)} dates intersected with by_date")
    if len(dates) < LOOKBACK + 1:
        print(f"Warning: {prefix}: not enough dates ({len(dates)})")
        return

    os.makedirs(out_dir, exist_ok=True)
    pending, max_tkr, sid = [], 0, 0
    feat_cols = FEAT_COLS  # cache locally

    for i in tqdm(range(LOOKBACK, len(dates) - 1), desc=f"build {prefix}"):
        win, nxt = dates[i - LOOKBACK:i], dates[i]
        if any(d not in by_date for d in (*win, nxt)):
            continue

        # Align rows (tickers) first
        mats  = [by_date[d][feat_cols] for d in win]
        rets  = by_date[nxt]["TARGET_RET"]
        common = set.intersection(*(set(m.index) for m in mats), set(rets.index))
        if not common:
            continue
        tickers = sorted(common)

        # Align columns and fill NA values
        mats = [
            m.loc[tickers].reindex(columns=feat_cols).astype(np.float32)
              .to_numpy(copy=False)
            for m in mats
        ]   # every mat now has shape (len(tickers), len(FEAT_COLS))

        cube = np.stack(mats)  # (L, N, F) — safe to stack
        y    = rets.loc[tickers].to_numpy(np.float32)

        # Mask invalid rows
        good = (~np.isnan(cube).any((0, 2))) & (~np.isnan(y))
        if not good.any():
            continue
        cube, y = cube[:, good, :], y[good]

        # Scale older days
        days = np.maximum([(win[-1] - d).days for d in win], 1)
        cube *= (15.0 / np.array(days, np.float32))[:, None, None]

        # Save un-padded file
        fp = os.path.join(out_dir, f"{prefix}_{sid:06d}.npz")
        np.savez(fp, X=cube, y=y)
        pending.append((fp, cube.shape[1]))
        max_tkr, sid = max(max_tkr, cube.shape[1]), sid + 1

    if sid == 0:
        print(f"Warning: {prefix}: zero samples created")
        return

    # Pad and compress files
    for fp, t in tqdm(pending, desc=f"pad {prefix}"):
        data = np.load(fp)
        X, y = data["X"], data["y"]
        L, F = X.shape[0], X.shape[2]

        padX = np.full((L, max_tkr, F), np.nan, np.float32)
        padX[:, :t] = X
        padY = np.full(max_tkr, np.nan, np.float32)
        padY[:t] = y
        mask = np.zeros(max_tkr, bool)
        mask[:t] = True

        np.savez_compressed(fp, X=padX, y=padY, m=mask)

    print(f"Success: {prefix}: {sid} samples → {out_dir} (max tickers={max_tkr})")


# Write train/test samples to disk
print("Writing train/test samples to disk")
make_samples_to_disk(train_dates, train_path, "train") 
make_samples_to_disk(test_dates,  test_path,  "test")
print("Script complete")


In [ ]:
import gc
import torch

def clear_torch_gpu_memory():
    # Delete any user-defined variables you no longer need
    # Example: del model, optimizer, X_tr, y_tr, m_tr, etc.

    # Run garbage collection
    gc.collect()

    # Empty the CUDA cache
    torch.cuda.empty_cache()

    # Reset peak memory stats if CUDA is available
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats()

# Example usage:
clear_torch_gpu_memory()
print("GPU memory after clear:", torch.cuda.memory_allocated(), "bytes allocated,",
      torch.cuda.memory_reserved(), "bytes reserved")


In [ ]:
# Robust REINFORCE implementation that streams samples from disk (.npz files)
# No changes to hyperparameters or model except for the I/O pipeline
import copy, time, math, glob, numpy as np, torch, torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import trange, tqdm

# Hyperparameters and configuration
USE_TX_COST   = False
TX_COST       = 0.019
EPOCHS        = 100
PATIENCE      = 10
BATCH         = 64
LR            = 1e-4
GAMMA         = 0.99
ENTROPY_BETA  = 1e-3
EMA_BETA      = 0.9
CLIP_GRAD     = 1.0
TOT_RET_RWD_SHARE = 0.9
device        = "cuda" if torch.cuda.is_available() else "cpu"

N_HEADS = 32
N_LAYERS = 20
if TESTING:
    N_HEADS, N_LAYERS, EPOCHS = 8, 4, 5

# Dataset class for loading .npz files from disk
class NPZDataset(Dataset):
    def __init__(self, root_dir: str):
        self.files = sorted(glob.glob(os.path.join(root_dir, "*.npz")))
        assert self.files, f"No .npz files found in {root_dir}"
        # Read one file to get dimensions
        sample = np.load(self.files[0])
        self.lookback, self.max_tkr, self.n_feat = sample["X"].shape

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        d = np.load(self.files[idx])
        X = torch.from_numpy(d["X"])
        y = torch.from_numpy(d["y"])
        m = torch.from_numpy(d["m"])
        return X, y, m

# Collate function for DataLoader
def collate(batch):
    X = torch.stack([b[0] for b in batch]).float()
    y = torch.stack([b[1] for b in batch]).float()
    m = torch.stack([b[2] for b in batch]).bool()
    return X.to(device), y.to(device), m.to(device)

# Initialize data loaders
train_ds = NPZDataset(train_path)
test_ds  = NPZDataset(test_path)
feat_dim = train_ds.n_feat

train_loader = DataLoader(train_ds, batch_size=BATCH,
                          shuffle=False, drop_last=False,
                          num_workers=0, collate_fn=collate)

# Policy network definition
class Policy(nn.Module):
    def __init__(self, feat_dim:int, d=128, h=N_HEADS,
                 layers=N_LAYERS, ff=256):
        super().__init__()
        self.proj = nn.Linear(feat_dim, d)
        enc_layer = nn.TransformerEncoderLayer(
            d, h, ff, 0.1, batch_first=True, activation="gelu")
        self.enc  = nn.TransformerEncoder(enc_layer, layers)
        self.head = nn.Linear(d, 1)

    def forward(self, x, mask):
        # x: (B,L,T,F)
        B, L, T, F = x.shape
        x = self.proj(torch.nan_to_num(x))              # (B,L,T,d)
        x = x.permute(0,2,1,3).reshape(B*T, L, -1)      # (B*T,L,d)
        h = self.enc(x)                                 # (B*T,L,d)
        logits = self.head(h).squeeze(-1)               # (B*T,L)
        logits = logits.view(B, T, L)[:, :, -1]         # (B,T)
        return logits.masked_fill(~mask, -1e9)

# Initialize model and optimizers
policy = Policy(feat_dim).to(device)
opt     = torch.optim.AdamW(policy.parameters(), lr=LR)
sched   = CosineAnnealingLR(opt, T_max=EPOCHS, eta_min=1e-5)

# Helper function to compute hard percentiles
def hard_percentile(z, mask):
    z = z.masked_fill(~mask, -1e9)
    rank = z.argsort(dim=1).argsort(dim=1).float()
    cnt  = mask.sum(dim=1, keepdim=True).clamp_min(1)
    return torch.where(cnt>1, rank/(cnt-1), torch.zeros_like(rank))

# Training loop
baseline, best_R, best_state = 0.0, -float("inf"), copy.deepcopy(policy.state_dict())
print(f"Device: {device} | Batches/epoch: {len(train_loader)}")

for ep in trange(1, EPOCHS+1, desc="Overall epochs"):
    policy.train()
    ep_rewards, ep_netrets, t0 = [], [], time.time()

    for Xb, yb, mb in tqdm(train_loader, desc=f"Epoch {ep}", leave=False):
        # Forward pass
        logits   = policy(Xb, mb)
        pct_sig  = hard_percentile(logits, mb)
        pct_ret  = hard_percentile(yb,    mb)

        # Calculate rewards
        align_r  = 1.0 - (pct_sig - pct_ret).abs().mean(dim=1)
        best_idx = logits.argmax(dim=1, keepdim=True)
        bonus_r  = yb.gather(1, best_idx).squeeze(1)

        # Calculate transaction costs if enabled
        cost = (TX_COST * pct_sig.abs().mean(dim=1)
                if USE_TX_COST else torch.zeros_like(align_r))

        # Combine rewards
        reward = (TOT_RET_RWD_SHARE * align_r +
                  (1-TOT_RET_RWD_SHARE) * bonus_r - cost)

        # Calculate discounted returns
        disc = GAMMA ** torch.arange(reward.size(0)-1, -1, -1, device=device)
        G    = (reward.flip(0).cumsum(0).flip(0)) / disc
        with torch.no_grad():
            baseline = EMA_BETA*baseline + (1-EMA_BETA)*G.mean().item()
        adv = G - baseline

        # Calculate loss
        logp    = F.log_softmax(logits, dim=1)
        logprob = (logp * pct_sig.detach()).sum(dim=1)
        entropy = -(logp.exp()*logp).sum(dim=1).mean()

        loss = -(logprob * adv.detach()).mean() - ENTROPY_BETA * entropy
        if torch.isnan(loss):
            continue                               # Skip pathological batch

        # Backward pass
        opt.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(policy.parameters(), CLIP_GRAD)
        opt.step()

        # Record metrics
        ep_rewards.append(reward.mean().item())
        ep_netrets.append((bonus_r - cost).mean().item())

    # Update learning rate and print metrics
    sched.step()
    avg_R   = float(np.mean(ep_rewards)) if ep_rewards else best_R
    avg_net = float(np.mean(ep_netrets)) if ep_netrets else 0.0
    print(f"Ep {ep:02d} | AvgReward {avg_R:+.6f}  AvgNetRet {avg_net:+.4%}  "
          f"LR {sched.get_last_lr()[0]:.5f}  Time {time.time()-t0:.1f}s")

    # Early stopping check
    if avg_R > best_R + 1e-5:
        best_R, best_state, epochs_no_improve = avg_R, copy.deepcopy(policy.state_dict()), 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= PATIENCE:
            print(f"Early stopping at epoch {ep}")
            break

# Load best model and finish
policy.load_state_dict(best_state)
print("Training complete. Best AvgReward:", best_R)


In [ ]:
# Save best model parameters locally
import torch
import os

# ensure directory exists
os.makedirs("models", exist_ok=True)

# path for saving
save_path = os.path.join("models", "best_policy_state.pt")

# save the state dict of the trained policy (best version)
torch.save(policy.state_dict(), save_path)

print(f"✔️ Best policy parameters saved to {save_path}")


In [ ]:
# Constants
WINDOW_LEN = 182  # six months of trading days
tb_daily = (1 + 0.03)**(1/252) - 1  # 3% annual T-bill rate converted to daily

import random
import matplotlib.pyplot as plt

# Helper functions for portfolio weight calculations
def pct_weights_np(vals: np.ndarray, mask: np.ndarray, frac=0.2) -> np.ndarray:
    # Calculate percentage weights based on value ranks
    vals_m = np.where(mask, vals, -1e9)
    ranks = np.argsort(np.argsort(vals_m, axis=1), axis=1).astype(float)
    cnt = mask.sum(axis=1, keepdims=True).clip(min=1)
    pct = ranks / (cnt - 1)
    long = pct >= (1 - frac)
    short = pct <= frac
    w = 0.5*long.astype(float) - 0.5*short.astype(float)
    return np.where(mask, w, 0.0)

def equal_weight(mask: np.ndarray) -> np.ndarray:
    # Calculate equal weights for all assets
    cnt = mask.sum(axis=1, keepdims=True).clip(min=1)
    return np.where(mask, 1.0 / cnt, 0.0)

def norm(arr: np.ndarray) -> np.ndarray:
    # Normalize array by its first value
    return arr / arr[0]

def load_window(ds: NPZDataset, start: int, length: int):
    # Load a contiguous window of samples from dataset
    Xs, ys, ms = [], [], []
    for j in range(start, start + length):
        X, y, m = ds[j]
        Xs.append(X)
        ys.append(y)
        ms.append(m)
    return (torch.stack(Xs),           # (length, L, T, F)
            torch.stack(ys),           # (length, T)
            torch.stack(ms))           # (length, T)

def plot_simulations_with_tb(n_sims: int = 5):
    # Main function to run and plot portfolio simulations
    for sim in range(1, n_sims + 1):
        # Select random 6-month window
        start_idx = random.randint(0, len(test_ds) - WINDOW_LEN - 1)
        if WINDOW_LEN <= LOOKBACK + 2:
            continue

        # Load data for simulation window
        Xw, yw, mw = load_window(test_ds, start_idx, WINDOW_LEN)
        y_np = yw.cpu().numpy()
        mask_np = mw.cpu().numpy().astype(bool)
        _, N_tkr = y_np.shape

        # Calculate portfolio weights for different strategies
        with torch.no_grad():
            logits = policy(Xw.to(device), mw.to(device)).cpu().numpy()
        w_learn = pct_weights_np(logits, mask_np)

        # Prior-day strategy weights
        prev_ret = np.vstack([np.zeros((1, N_tkr)), y_np[:-1]])
        w_prev = pct_weights_np(prev_ret, mask_np)

        # EWMA strategy weights
        ewm_ret = (pd.DataFrame(y_np)
                  .ewm(alpha=0.98, adjust=False)
                  .mean().shift(1).fillna(0).values)
        w_ewma = pct_weights_np(ewm_ret, mask_np)

        # Equal weight strategy
        w_eq = equal_weight(mask_np)

        # Portfolio simulation function
        def simulate(w_mat):
            ret_raw = np.nansum(w_mat * y_np, axis=1)
            w_shift = np.vstack([np.zeros((1, N_tkr)), w_mat[:-1]])
            turnover = np.abs(w_mat - w_shift).sum(axis=1)
            ret_net = ret_raw - TX_COST * turnover
            pv_raw = norm(np.cumprod(1 + ret_raw))
            pv_net = norm(np.cumprod(1 + ret_net))
            return pv_raw, pv_net, ret_raw, ret_net

        # Run simulations for each strategy
        pv_Lr, pv_Ln, ret_Lr, ret_Ln = simulate(w_learn)
        pv_Pr, pv_Pn, ret_Pr, ret_Pn = simulate(w_prev)
        pv_Er, pv_En, ret_Er, ret_En = simulate(w_ewma)
        pv_EQr, pv_EQn, ret_EQr, ret_EQn = simulate(w_eq)

        # Calculate T-bill benchmark
        pv_tb = norm((1 + tb_daily) ** np.arange(1, len(pv_Lr) + 1))
        ret_tb = np.full_like(ret_Lr, tb_daily)

        # Calculate Sharpe ratios
        def sharpe(ret_series):
            excess = ret_series - tb_daily
            return (excess.mean() / excess.std()) * np.sqrt(252)

        sr = {
            "Learned": sharpe(ret_Lr),
            "Prior-day": sharpe(ret_Pr),
            "EWMA": sharpe(ret_Er),
            "Equal-weight": sharpe(ret_EQr),
        }
        sr_net = {
            "Learned": sharpe(ret_Ln),
            "Prior-day": sharpe(ret_Pn),
            "EWMA": sharpe(ret_En),
            "Equal-weight": sharpe(ret_EQn),
        }

        days = np.arange(len(pv_Lr))
        start_lbl = f"{start_idx}"
        end_lbl = f"{start_idx + WINDOW_LEN - 1}"

        # Plot A: Performance without costs
        fig, ax = plt.subplots(figsize=(8, 3), dpi=300)
        ax.plot(days, pv_Lr, label=f"Learned (SR={sr['Learned']:.2f})", color="tab:blue")
        ax.plot(days, pv_Pr, label=f"Prior-day (SR={sr['Prior-day']:.2f})", color="tab:red")
        ax.plot(days, pv_Er, label=f"EWMA (SR={sr['EWMA']:.2f})", color="tab:orange")
        ax.plot(days, pv_EQr, label=f"Equal-weight (SR={sr['Equal-weight']:.2f})", color="purple")
        ax.plot(days, pv_tb, label="T-bill 3 %", color="tab:green", ls="--")
        ax.set_title(f"Sim {sim} {start_lbl}→{end_lbl}  (No Costs)")
        ax.set_xlabel("Day")
        ax.set_ylabel("Normalized PV")
        ax.legend(loc="center left", bbox_to_anchor=(1.02, 0.5), frameon=False)
        fig.tight_layout(rect=[0,0,0.8,1])
        fig.savefig(f"./eda_plots/simulation_{sim}_no_cost_sr.png")
        plt.show()

        # Plot B: Performance with costs
        fig, ax = plt.subplots(figsize=(8, 3), dpi=300)
        ax.plot(days, pv_Ln, label=f"Learned (SR={sr_net['Learned']:.2f})", color="tab:blue")
        ax.plot(days, pv_Pn, label=f"Prior-day (SR={sr_net['Prior-day']:.2f})", color="tab:red")
        ax.plot(days, pv_En, label=f"EWMA (SR={sr_net['EWMA']:.2f})", color="tab:orange")
        ax.plot(days, pv_EQn, label=f"Equal-weight (SR={sr_net['Equal-weight']:.2f})", color="purple")
        ax.plot(days, pv_tb, label="T-bill 3 %", color="tab:green", ls="--")
        ax.set_title(f"Sim {sim} {start_lbl}→{end_lbl}  (With Costs)")
        ax.set_xlabel("Day")
        ax.set_ylabel("Normalized PV")
        ax.legend(loc="center left", bbox_to_anchor=(1.02, 0.5), frameon=False)
        fig.tight_layout(rect=[0,0,0.8,1])
        fig.savefig(f"./eda_plots/simulation_{sim}_with_cost_sr.png")
        plt.show()

        # Plot C: Combined raw vs net performance
        fig, ax = plt.subplots(figsize=(8, 3), dpi=300)
        for label, col in [
            ("Learned", "tab:blue"),
            ("Prior-day", "tab:red"),
            ("EWMA", "tab:orange"),
            ("Equal-weight", "purple"),
        ]:
            ax.plot(days,
                    {"Learned": pv_Lr, "Prior-day": pv_Pr, "EWMA": pv_Er, "Equal-weight": pv_EQr}[label],
                    label=f"{label} Raw (SR={sr[label]:.2f})",
                    color=col)
            ax.plot(days,
                    {"Learned": pv_Ln, "Prior-day": pv_Pn, "EWMA": pv_En, "Equal-weight": pv_EQn}[label],
                    label=f"{label} Net (SR={sr_net[label]:.2f})",
                    color=col, alpha=0.4)
        ax.plot(days, pv_tb, label="T-bill 3 %", color="tab:green", ls="--")
        ax.set_title(f"Sim {sim} {start_lbl}→{end_lbl}  (Raw vs Net All)")
        ax.set_xlabel("Day"); ax.set_ylabel("Normalized PV")
        ax.legend(loc="center left", bbox_to_anchor=(1.02, 0.5), frameon=False)
        fig.tight_layout(rect=[0,0,0.8,1])
        fig.savefig(f"./eda_plots/simulation_{sim}_combined_all_sr.png")
        plt.show()

# run the simulations with Sharpe-annotated plots
plot_simulations_with_tb(5)


In [ ]:
# Simulation code for portfolio strategies with and without trading costs
# Includes functions for weight calculation and simulation, plus plotting utilities

import os, random
import numpy as np, torch, pandas as pd, matplotlib.pyplot as plt

# Create directory for plots
os.makedirs("./eda_plots", exist_ok=True)

# Constants
WINDOW_LEN = 182  # six-month window length
tb_daily = (1 + 0.03)**(1/252) - 1  # daily T-bill rate

def pct_weights_np(vals, mask, frac=0.2):
    # Calculate percentage weights based on value ranks
    vals_m = np.where(mask, vals, -1e9)
    ranks = np.argsort(np.argsort(vals_m, axis=1), axis=1).astype(float)
    cnt = mask.sum(axis=1, keepdims=True).clip(min=1)
    pct = ranks/(cnt - 1)
    long = pct >= (1 - frac)
    short = pct <= frac
    return np.where(mask, 0.5*long - 0.5*short, 0.0)

def simulate_once(start_idx):
    # Run one simulation with different portfolio strategies
    Xw, yw, mw = load_window(test_ds, start_idx, WINDOW_LEN)
    y_np = yw.cpu().numpy()
    m_np = mw.cpu().numpy().astype(bool)

    # Learned strategy weights
    with torch.no_grad():
        logits = policy(Xw.to(device), mw.to(device)).cpu().numpy()
    w_learn = pct_weights_np(logits, m_np)

    # Prior-day strategy weights
    prev_ret = np.vstack([np.zeros((1, y_np.shape[1])), y_np[:-1]])
    w_prev = pct_weights_np(prev_ret, m_np)

    # EWMA strategy weights
    ewm = (pd.DataFrame(y_np)
           .ewm(alpha=0.98, adjust=False).mean()
           .shift(1).fillna(0).values)
    w_ewma = pct_weights_np(ewm, m_np)

    # Equal-weight strategy
    cnt = m_np.sum(axis=1, keepdims=True).clip(min=1)
    w_eq = np.where(m_np, 1.0 / cnt, 0.0)

    def sim_pv(w):
        # Calculate portfolio value with and without trading costs
        ret_raw = np.nansum(w * y_np, axis=1)
        w0 = np.vstack([np.zeros((1, w.shape[1])), w[:-1]])
        trn = np.abs(w - w0).sum(axis=1)
        ret_net = ret_raw - TX_COST * trn
        return np.cumprod(1 + ret_raw), np.cumprod(1 + ret_net), ret_raw, ret_net

    return sim_pv(w_learn), sim_pv(w_prev), sim_pv(w_ewma), sim_pv(w_eq)

# Run 10 simulations
n_sims = 10
strats = ["learn", "prior", "ewma", "eq"]
all_no, all_wc, ret_no_raw, ret_wc_raw, lengths = \
    {k: [] for k in strats}, {k: [] for k in strats}, \
    {k: [] for k in strats}, {k: [] for k in strats}, []

for _ in range(n_sims):
    start_idx = random.randint(0, len(test_ds) - WINDOW_LEN - 1)
    (lr, ln, rr_Lr, rr_Ln), \
    (pr, pn, rr_Pr, rr_Pn), \
    (er, en, rr_Er, rr_En), \
    (qr, qn, rr_EQr, rr_EQn) = simulate_once(start_idx)

    # Store results for each strategy
    for k, arr in zip(strats, [lr, pr, er, qr]):
        all_no[k].append(arr)
    for k, arr in zip(strats, [ln, pn, en, qn]):
        all_wc[k].append(arr)

    for k, r in zip(strats, [rr_Lr, rr_Pr, rr_Er, rr_EQr]):
        ret_no_raw[k].append(r)
    for k, r in zip(strats, [rr_Ln, rr_Pn, rr_En, rr_EQn]):
        ret_wc_raw[k].append(r)

    lengths.append(len(lr))

# Pad arrays to common length and normalize to starting value of 1
max_len = max(lengths)
def pad_and_norm(lst):
    A = np.zeros((len(lst), max_len))
    for i, a in enumerate(lst):
        L = len(a)
        A[i, :L] = a
        A[i, L:] = a[-1]
    return A / A[:, [0]]

no_pad = {k: pad_and_norm(v) for k, v in all_no.items()}
wc_pad = {k: pad_and_norm(v) for k, v in all_wc.items()}

# Pad returns arrays (without normalization)
def pad_ret(lst):
    A = np.zeros((len(lst), max_len - 1))
    for i, a in enumerate(lst):
        L = len(a)
        r = a[1:] / a[:-1] - 1  # Calculate returns
        LL = len(r)
        A[i, :LL] = r
        A[i, LL:] = 0.0
    return A

ret_no = {k: pad_ret(ret_no_raw[k]) for k in strats}
ret_wc = {k: pad_ret(ret_wc_raw[k]) for k in strats}

# Calculate Sharpe ratios for each strategy
sr_no = {}
for k in strats:
    ex = ret_no[k] - tb_daily
    sr_sim = (ex.mean(axis=1) / ex.std(axis=1)) * np.sqrt(252)
    sr_no[k] = float(sr_sim.mean())

sr_wc = {}
for k in strats:
    ex = ret_wc[k] - tb_daily
    sr_sim = (ex.mean(axis=1) / ex.std(axis=1)) * np.sqrt(252)
    sr_wc[k] = float(sr_sim.mean())

# Plotting parameters
days = np.arange(1, max_len + 1)
tb_norm = ((1 + tb_daily) ** days) / ((1 + tb_daily) ** days)[0]
alpha_value = 0.01
sample_size = 1000

# Plot 1: No Trading Costs (without Sharpe ratios)
plt.figure(figsize=(8, 4), dpi=300)
ax = plt.gca()
for k, col in zip(strats, ["tab:blue", "tab:red", "tab:orange", "purple"]):
    M = no_pad[k]
    draw = min(sample_size, M.shape[0])
    for arr in M[np.random.choice(M.shape[0], draw, replace=False)]:
        ax.plot(days, arr, color=col, alpha=max(alpha_value, 1 / n_sims))
    ax.plot(days, M.mean(0), color=col, lw=2, label=f"{k.title()} mean")
ax.plot(days, tb_norm, color="tab:green", ls="--", label="T-bill 3%")
ax.set_title("Normalized PV — No Trading Costs")
ax.set_xlabel("Day"); ax.set_ylabel("Normalized PV")
ax.legend(loc="center left", bbox_to_anchor=(1.01, 0.5), frameon=False)
plt.tight_layout(rect=[0, 0, 0.85, 1])
plt.savefig("./eda_plots/10_simulations_normalized_no_cost.png")
plt.show()

# Plot 2: No Trading Costs (with Sharpe ratios)
plt.figure(figsize=(8, 4), dpi=300)
ax = plt.gca()
for k, col in zip(strats, ["tab:blue", "tab:red", "tab:orange", "purple"]):
    M = no_pad[k]
    draw = min(sample_size, M.shape[0])
    for arr in M[np.random.choice(M.shape[0], draw, replace=False)]:
        ax.plot(days, arr, color=col, alpha=max(alpha_value, 1 / n_sims))
    ax.plot(days, M.mean(0), color=col, lw=2,
            label=f"{k.title()} mean (SR={sr_no[k]:.2f})")
ax.plot(days, tb_norm, color="tab:green", ls="--", label="T-bill 3%")
ax.set_title("Normalized PV — No Trading Costs (w/Sharpe)")
ax.set_xlabel("Day"); ax.set_ylabel("Normalized PV")
ax.legend(loc="center left", bbox_to_anchor=(1.01, 0.5), frameon=False)
plt.tight_layout(rect=[0, 0, 0.85, 1])
plt.savefig("./eda_plots/10_simulations_normalized_no_cost_sr.png")
plt.show()

# ─────────────── PLOT: With Trading Costs (no SR) ──────────────────────────
plt.figure(figsize=(8, 4), dpi=300)
ax = plt.gca()
for k, col in zip(strats, ["tab:blue", "tab:red", "tab:orange", "purple"]):
    M = wc_pad[k]
    draw = min(sample_size, M.shape[0])
    for arr in M[np.random.choice(M.shape[0], draw, replace=False)]:
        ax.plot(days, arr, color=col, alpha=max(alpha_value, 1 / n_sims))
    ax.plot(days, M.mean(0), color=col, lw=2, label=f"{k.title()} mean")
ax.plot(days, tb_norm, color="tab:green", ls="--", label="T-bill 3%")
ax.set_title("Normalized PV — With Trading Costs")
ax.set_xlabel("Day"); ax.set_ylabel("Normalized PV")
ax.legend(loc="center left", bbox_to_anchor=(1.01, 0.5), frameon=False)
plt.tight_layout(rect=[0, 0, 0.85, 1])
plt.savefig("./eda_plots/10_simulations_normalized_with_cost.png")
plt.show()

# ─────────────── PLOT: With Trading Costs (with SR) ────────────────────────
plt.figure(figsize=(8, 4), dpi=300)
ax = plt.gca()
for k, col in zip(strats, ["tab:blue", "tab:red", "tab:orange", "purple"]):
    M = wc_pad[k]
    draw = min(sample_size, M.shape[0])
    for arr in M[np.random.choice(M.shape[0], draw, replace=False)]:
        ax.plot(days, arr, color=col, alpha=max(alpha_value, 1 / n_sims))
    ax.plot(days, M.mean(0), color=col, lw=2,
            label=f"{k.title()} mean (SR={sr_wc[k]:.2f})")
ax.plot(days, tb_norm, color="tab:green", ls="--", label="T-bill 3%")
ax.set_title("Normalized PV — With Trading Costs (w/Sharpe)")
ax.set_xlabel("Day"); ax.set_ylabel("Normalized PV")
ax.legend(loc="center left", bbox_to_anchor=(1.01, 0.5), frameon=False)
plt.tight_layout(rect=[0, 0, 0.85, 1])
plt.savefig("./eda_plots/10_simulations_normalized_with_cost_sr.png")
plt.show()
